In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
submission_df = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

test_df = test_df[['excerpt']]

In [ ]:
test_df.head()

In [ ]:
import transformers
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup,  AutoConfig
import sentencepiece

import torch
import numpy as np
import pandas as pd


import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


%matplotlib inline
%config InlineBackend.figure_format='retina'


sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import gc

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained('../input/roberta-base')

In [ ]:
class CommonLitDataset(Dataset):
  def __init__(self, text, targets = None, tokenizer = transformers.RobertaTokenizer.from_pretrained('../input/roberta-base'), max_len = 256, inference_only = False):
    self.text = text
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.inference_only = inference_only
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, item):
    text = str(self.text[item])
    if self.inference_only == False:
        targets = self.targets[item]
    
    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      truncation=True,
      padding = 'max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )
    if self.inference_only == True:
        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
        }
    else:
        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(targets, dtype=torch.float)
        }

In [ ]:
MAX_LEN = 256

In [ ]:
def create_data_loader(df, tokenizer, max_len=256, batch_size=8):
  ds = CommonLitDataset(
    text = df.excerpt.to_list(),
    tokenizer=tokenizer,
    max_len=max_len,
    inference_only = True
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
class ReadabilityModel(nn.Module):
  def __init__(self):

    super(ReadabilityModel, self).__init__()

    config = AutoConfig.from_pretrained('../input/roberta-base')
    config.update(
        {
            'output_hidden_states':True,
            'hidden_dropout_prob' : 0.0,
            'layer_norm_eps': 1e-7
        }
    )


    self.roberta = RobertaForSequenceClassification.from_pretrained('../input/roberta-base', config = config)

    self.attention = nn.Sequential(
        nn.Linear(768, 512),
        nn.Tanh(),
        nn.Linear(512,1),
        nn.Softmax(dim=1)
    )
    self.regressor = nn.Sequential(
        nn.Linear(768,1)
    )
    

  def forward(self, input_ids, attention_mask):
    output = self.roberta(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    last_layer_hidden_states = output.hidden_states[-1]
    
    weights = self.attention(last_layer_hidden_states)

    context_vector = torch.sum(weights*last_layer_hidden_states, dim = 1)

    return self.regressor(context_vector)

In [ ]:
def predict(model, data_loader):
    model.eval()
    
    results = np.zeros(len(data_loader.dataset))
    
    index = 0
    
    with torch.no_grad():
        for batch_num, data in enumerate(data_loader):
            input_ids  = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            
            pred = model(input_ids, attention_mask)
            
            results[index: index+pred.shape[0]] = pred.flatten().to('cpu')
            index += pred.shape[0]
            
    return results

In [ ]:
NUM_MODELS = 3
all_predictions = np.zeros((NUM_MODELS,len(test_df)))

test_dataloader = create_data_loader(test_df, tokenizer = tokenizer)


                        
model = ReadabilityModel()
model.load_state_dict(torch.load('../input/commonlitweights/3.bin', map_location=device))    
model.to(device)
        
all_predictions[0] = predict(model, test_dataloader)
            
del model
gc.collect()


model = ReadabilityModel()
model.load_state_dict(torch.load('../input/commonlitweights/model-train.bin', map_location=device))    
model.to(device)
        
all_predictions[1] = predict(model, test_dataloader)
            
del model
gc.collect()


model = ReadabilityModel()
model.load_state_dict(torch.load('../input/commonlitweights/model.bin', map_location=device))    
model.to(device)
        
all_predictions[2] = predict(model, test_dataloader)
            
del model
gc.collect()



In [ ]:
all_predictions

In [ ]:
final_pred = all_predictions[0]*.4 + all_predictions[1]*.2 + all_predictions[2]*.4

In [ ]:
final_pred

In [ ]:
submission_df['target'] = final_pred
print(submission_df)
submission_df.to_csv('submission.csv',index = False)